# Adversarial Search: Playing Connect 4


## Instructions

Total Points: Undegraduates 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play Connect 4:

> "Connect 4 is a two-player connection board game, in which the players choose a color and then take turns dropping colored discs into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs." (see [Connect Four on Wikipedia](https://en.wikipedia.org/wiki/Connect_Four))

## Task 1: Defining the Search Problem [1 point]

Define the components of the search problem:

<b>Initial state</b>: The initial state is the state the agent starts in (Russel 64). In the context of a game, a state, <i>s</i>, can be defined as a function of (position, board). For Connect Four, the initial state will be written as <i>s0</i>, where s0 = empty board. Using the earlier described function notation, the game can also be written as (position=none, board=empty).

<b>Actions</b>: Given a state <i>s</i>, actions(s) are the finite set of options that can be executed in this state <i>s</i> (Russel 64). In terms of Connect Four, the actions are playing in one of the empty spaces (actions are everywhere the player can move). In Connect Four, there is one valid action/move/play per column.

<b>Transition model</b>: 
A transition model is a way to describe what each action does. In other words, transition models maps a state and action to a resulting state. The environment of ConnectFour is non-deterministic, and a transition model is needed to describe the uncertainty and results of each action. The transition model will return the resulting state from the action <i>a</i>. The transition model will return the board after the piece has been played at the location designated by the agent. The transition model returns the result of an agent's action from their turn. 

<b>Goal State</b>:  The goal state is the desired state of the agent. For Connect Four, the goal state is victory achieved by having 4 pieces in a row either horizontally, vertically, or diagonally. Both a loss or a draw are considered a failure and not apart of the goal state. 

How big is the search space?

While the exact state space/search space of Conncet Four is unknown, we can do a quick estimation to get an idea of the state space. 

We will model the enviornment as a permutation (Similar to the class handout: "how many ways can we order/arrange the objects in the environment"). Each space has three possible options:  "x piece", "y piece", and empty (n == 3). The total number of slots is 42 because the board size is 6x7 (r == 42). However we know that not all these states might be reachable because the path leading to them would result in a terminal state/ end the game. Therefore the real state space will be smaller than the theoritical calculation 

In [1]:
#permutation state space calcualtion as explained above: 3^42 (3 = r, n = 32)
3**42

109418989131512359209

Therefor, the actual state space is  "< 109418989131512359209"

__Note:__ The search space for a $6 \times 7$ board is large. You can experiment with smaller boards (the smallest is $4 times \4$) and/or changing the winning rule to connect 3 instead of 4.

## Task 2: Game Environment and Random Agent [2 point]

Use a numpy character array as the board.

Instead of colors for the players use 'x' and 'o' to represent the players. Make sure that your agent functions all have the from: `agent_type(board, player = 'x')`, where board is the current board position and player is the player whose next move it is and who the agent should play.

Implement the board and helper functions for:

* The transition model (result).
* The utility function.
* Check for terminal states.
* A check for available actions.
* A function to visualize the board.

Make sure that all these functions work with boards of different sizes.

Implement an agent that plays randomly and let two random agents play against each other 1000 times. How often does each player win? Is the result expected? 

<b>Helper Functions:</b>

In [2]:
#import statements:
import numpy as np
import random
#global declarations:
shape=(6, 7)
winCondition = 4 #how many pieces connected in a row to win
player1Score = 0
player1Wins = 0 #total number of wins
player2Score = 0
player2Wins = 0
draws = 0

#Use a numpy character array as the board.
def empty_board(): 
    global shape
    return np.full(shape=shape, fill_value=' ')

#helper function determining if a space is valid or not
def isValid(x,y):
    global shape
    if(x >= 0 and y >=0):
        if (x <= shape[0] - 1 and y <= shape[1] - 1):
            return True
    return False

#TERMINAL STATES: test for terminal states, or win conditions, that would end the game:
def terminalStates(board):
    arr = ["right", "up", "down", "left", "diagnoalUpRight", "diagnoalUpLeft", "diagnoalLowLeft", "diagnoalLowRight"]
    global winCondition
    for i in range(0, len(board)):
        row = board[i]
        for j in range(0, len(row)):  # for each space
            currSpace = board[i][j]
            if (currSpace != 'x' and currSpace != 'o'):
                continue
            #each direction from current valid space
            for direction in arr:
                # check to the right:
                counter = 1
                colPos = j
                rowPos = i
                while (counter <= winCondition):
                    #----- determine what direction -----
                    if(direction == "right"):
                        colPos = colPos + 1
                    if(direction == "left"):
                        colPos = colPos - 1
                    if(direction == "up"):
                        rowPos = rowPos + 1
                    if(direction == "down"):
                        rowPos = rowPos - 1
                    # move in a diagnol direction
                    if(direction == "diagnoalUpRight"):
                        colPos = colPos + 1
                        rowPos = rowPos + 1
                    if(direction == "diagnoalUpLeft"):
                        colPos = colPos + 1
                        rowPos = rowPos - 1
                    if(direction == "diagnoalLowRight"):
                        rowPos = rowPos - 1
                        colPos = colPos + 1
                    if(direction == "diagnoalLowLeft"):
                        rowPos = rowPos - 1
                        colPos = colPos - 1
                    #----- end direction ---------
                    if (isValid(rowPos, colPos)):  # if space exists
                        if (board[rowPos][colPos] == currSpace):  # continuing with valid win condition
                            counter = counter + 1
                            if (counter == winCondition): #win condition reached
                                return currSpace
                            continue
                        else:
                            break
                    else:
                        break
    #---- end for loop ----
    return 'none'


#return a List of all the available moves for a player in the environment. 
def availableActions(board):
    validSpaces = []
    for i in range(0, shape[1]): #iterate through columns -> 1 valid move per column (typically)
        for j in range(shape[0] - 1, -1, -1):
            currSpace = board[j][i]
            if (currSpace != 'x' and currSpace != 'o'):
                location = (j, i)
                validSpaces.append(location)
                break
    return validSpaces


#print the board to the screen# transition model models the results of an action: If I place token, how does the board look now
def visualizeBoard(board):
    print(board)
    
#assign the winner. +1 for a win, 0 for a draw, -1 for a loss
def utilityFunction(player):
    # keep track of total wins
    global player1Wins
    global player2Wins
    #
    global player2Score
    global player1Score
    global draws
    if('x' == player):
        player1Score = player1Score + 1
        player2Score -= 1
        player1Wins += 1
    elif('o' == player):
        player1Score -= 1
        player2Score += 1
        player2Wins += 1
    else:
        draws += 1
    #----- end function ----

#switch player
def switch_player(player):
    if player == 'x':
        return 'o'
    else:
        return 'x'

#clear scores from the utility function:
def clearScores():
    global player1Score
    global player2Score
    global player1Wins
    global player2Wins
    global draws
    player1Score = player2Score = 0
    player1Wins = player2Wins = 0
    draws = 0
    
    
# transition model models the results of an action: If I place token, how does the board look now 
def transitionModel(board, location, player):
    try:
        board[location[0]][location[1]] = player
    except:
        print("error, check indexing")
        

#statically evaluate a space
# statically evaluate a space
def evalSpace(board, space, currPlayer):
    arr = ["right", "up", "down", "left", "diagnoalUpRight", "diagnoalUpLeft", "diagnoalLowLeft", "diagnoalLowRight"]
    maxCount = 0
    for direction in arr:
        rowPos = space[0]
        colPos = space[1]
        count = 0
        for x in range(0, winCondition):
            # ----- determine what direction -----
            if (direction == "right"):
                colPos = colPos + 1
            if (direction == "left"):
                colPos = colPos - 1
            if (direction == "down"):
                rowPos = rowPos + 1
            if (direction == "up"):
                rowPos = rowPos - 1
            # move in a diagnol direction
            if (direction == "diagnoalUpRight"):
                colPos = colPos + 1
                rowPos = rowPos + 1
            if (direction == "diagnoalUpLeft"):
                colPos = colPos + 1
                rowPos = rowPos - 1
            if (direction == "diagnoalLowRight"):
                rowPos = rowPos - 1
                colPos = colPos + 1
            if (direction == "diagnoalLowLeft"):
                rowPos = rowPos - 1
                colPos = colPos - 1
            # ----- end direction ---------
            count = count + 1
            #if space is invalid breakout of loop
            if (not isValid(rowPos, colPos)):
                if (count > maxCount):
                    maxCount = count
                break
            currSpace = board[rowPos][colPos]
            # if not the current player, breakout of the loop
            if (currSpace != currPlayer):
                if (count > maxCount):
                    maxCount = count
                break
    return maxCount


<b>Random Agent Implementation:</b>

In [3]:
#Implement an agent that plays randomly and let two random agents play against each other 1000 times. 
#How often does each player win? Is the result expected? 
def agent_random(board, player):
    spaces = availableActions(board)
    if(len(spaces) == 0):
        return spaces
    return random.choice(spaces)

In [4]:
def play_random(n = 1000):
    player1 = 'x'
    player2 = 'o'
    for i in range(0,n):
        currPlayer = player1
        board = empty_board()
        result = "none"
        while(result == "none"):
            myChoice = agent_random(board, currPlayer)
            if(len(myChoice) < 2):
                break
            transitionModel(board, myChoice, currPlayer)
            #switch players:
            if currPlayer == player1:
                currPlayer = player2
            else:
                currPlayer = player1
            result = terminalStates(board)
        utilityFunction(result)
        #visualizeBoard(board)
    #output results to the console
    print('player 1 wins: ', player1Wins)
    print('player 2 wins: ', player2Wins)
    print('total draws:', draws)
    clearScores()
    

In [5]:
play_random()

player 1 wins:  562
player 2 wins:  435
total draws: 3


The results of 1,000 trials of two randow agents are as follows: player 1 winning 540 times and player 2 winning 457 times out of a thousand. These results are as expected because it is close to 50/50 or random chance. Player 1 has a slight advantage to player2. This discrepency may be attributed to player 1 always moving first. Always moving first means player 1 has a slighty higher chance of winning all its games when compared with player 2 always moving second. Overall, the results make logical sesne and are understandable. 

## Task 3: Minimax Search with Alpha-Beta Pruning [4 points]

### Implement the search starting from a given board and specifying the player.

In [6]:
#minimax search:
#PARAMATERS:
# alpha = lower bound, beta = upperbound
# board, space = state information. space is a pair
# currPlayer = 'x' , 'o'
# maximizingPlayer = True, False (maxmize if True, minimize if false)

def minimax(board, currPlayer, depth):
    #alpha = +infinity. beta = -infinity
    space = (0,0) #space = initial starting space
    myEval = maxValue(board, space, currPlayer, depth, 1000, -1000)
    #modify 3 to MAX value for non-depth resitrected (board too large to do this right nwo)
    return myEval[1]

#FUNCTION RETURNING MAXIMUM VALUE
def maxValue(board, space, currPlayer, depth, alpha, beta):
    #if terminal states OR depth is reached --> terminate
    result = terminalStates(board)
    if(depth == 0 or result != "none"): #if terminal states:
        evalResult = (evalSpace(board, space, currPlayer), space)
        return evalResult

    # actual maximum evaluation:
    maxEval = -1  #intilize to negative infinity
    maxMove = (0,0)
    children = availableActions(board)
    for child in children:
        evaluation = minValue(board, child, switch_player(currPlayer), (depth - 1), alpha, beta )
        #evaluation[0] = stativEvaluation of how good a move is
        #evaluation[1] = actual move / space coordiantes
        if(maxEval < evaluation[0]): #if new value is greater than maxVolume
            maxEval = evaluation[0]
            maxMove = evaluation[1]
            alpha = max(alpha, maxEval)
        if(maxEval <= beta):
            x = 1
            break
    return (maxEval, maxMove)

#FUNCTION RETURNING MINMUM VALUE
def minValue(board, space, currPlayer, depth, alpha, beta):
    # --- if terminal states OR depth is reached ---
    result = terminalStates(board)
    if(depth == 0 or result != "none"): #if terminal states:
        evalResult = (evalSpace(board, space, switch_player(currPlayer) ), space)
        return evalResult

    # actual min evaluation:
    minEval = 1000
    minMove = (0,0)
    children = availableActions(board)
    for child in children:
        evaluation = maxValue(board, child, switch_player(currPlayer), (depth - 1), alpha, beta )
        #evaluation[0] = stativEvaluation of how good a move is
        #evaluation[1] = space coordiantes
        if(minEval > evaluation[0]): #if new value is less than maximum value
            minEval = evaluation[0]
            minMove = evaluation[1]
            beta = min(beta, minEval)
        if(minEval >= alpha):
            x = 1
            break
    return (minEval, minMove)

Experiment with some manually created boards (at least 5) to check if the agent spots wining opportunities.

Board 1: Looking right/left to find solution. Agent finds winning move (5,3) to get 4 x's in a row. Causes terminal state.

In [7]:
#do minimax:
board = empty_board()
board[5][0] = board[5][1] = board[5][2] = 'x'
move = minimax(board, 'x', 5)
#output results:
visualizeBoard(board)
print(move)

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['x' 'x' 'x' ' ' ' ' ' ' ' ']]
(5, 3)


Board# 2a: Looking up/down to find solution. Agent finds winning move (2,0) to get 4 x's in a vertical column. Causes terminal state.

In [8]:
board = empty_board()
board[5][0] = board[4][0] = board[3][0] = 'x'
board[5][3] = board[4][3] = board[3][3] = 'o' 
move = minimax(board, 'x', 5)
#output results:
visualizeBoard(board)
print(move)

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['x' ' ' ' ' 'o' ' ' ' ' ' ']
 ['x' ' ' ' ' 'o' ' ' ' ' ' ']
 ['x' ' ' ' ' 'o' ' ' ' ' ' ']]
(2, 0)


Board 2b: Looking up/down to find solution for 'o'. Agent finds winning move (2,3) to get 4 o's in a vertical column. Causes terminal state.

In [9]:
board = empty_board()
board[5][0] = board[4][0] = board[3][0] = 'x'
board[5][3] = board[4][3] = board[3][3] = 'o' 
move = minimax(board, 'o', 5)
#output results:
visualizeBoard(board)
print(move)

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['x' ' ' ' ' 'o' ' ' ' ' ' ']
 ['x' ' ' ' ' 'o' ' ' ' ' ' ']
 ['x' ' ' ' ' 'o' ' ' ' ' ' ']]
(2, 3)


Board 3: Looking diaganol to find solution for 'x'. Agent finds winning move (5,3) to get 4 x's in a diagnol line. Causes terminal state.

In [10]:
#intilize board:
board = empty_board()
board[2][0] = board[3][1] = board[4][2] = 'x'
board[5][0] = board[4][0] = board[3][0] = 'o' 
board[5][1] = board[4][1] = board[5][2] = 'o' 
#output results:
move = minimax(board, 'x', 5)
visualizeBoard(board)
print(move)

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['x' ' ' ' ' ' ' ' ' ' ' ' ']
 ['o' 'x' ' ' ' ' ' ' ' ' ' ']
 ['o' 'o' 'x' ' ' ' ' ' ' ' ']
 ['o' 'o' 'o' ' ' ' ' ' ' ' ']]
(5, 3)


Board 4: Looking right/left to find solution for 'o'. Agent finds winning move (5,3) to get 4 x's in a line. Causes terminal state.

In [11]:
#intilize board:
board = empty_board()
board[2][0] = board[3][1] = board[4][2] = 'x'
board[5][0] = board[4][0] = board[3][0] = 'o' 
board[5][1] = board[4][1] = board[5][2] = 'o' 
#output results:
move = minimax(board, 'o', 5)
visualizeBoard(board)
print(move)

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['x' ' ' ' ' ' ' ' ' ' ' ' ']
 ['o' 'x' ' ' ' ' ' ' ' ' ' ']
 ['o' 'o' 'x' ' ' ' ' ' ' ' ']
 ['o' 'o' 'o' ' ' ' ' ' ' ' ']]
(5, 3)


Board 5: Agent finds winning move (2,6) to get 4 o's in a vertical column. Causes terminal state.

In [12]:
#intilize board:
board = empty_board()
board[5][3] = board[4][4] = board[3][5] ='x'
board[5][6] = board[4][6] = board [3][6] = 'o'

#output results:
move = minimax(board, 'o', 5)
visualizeBoard(board)
print(move)

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'x' 'o']
 [' ' ' ' ' ' ' ' 'x' ' ' 'o']
 [' ' ' ' ' ' 'x' ' ' ' ' 'o']]
(2, 6)


How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [28]:
%%time
# Your code/ answer goes here.
shape=(4, 4)
winCondition = 3 #how many pieces connected in a row to win


board = empty_board()
move = minimax(board, 'x', 10)
#output results:
visualizeBoard(board)
print(move)

[[' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']]
(3, 0)
Wall time: 30.3 s


In [29]:
%%time
# Your code/ answer goes here.
shape=(5, 5)
winCondition = 3 #how many pieces connected in a row to win


board = empty_board()
move = minimax(board, 'x', 10)
#output results:
visualizeBoard(board)
print(move)

[[' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
(4, 0)
Wall time: 5min 40s


In [33]:
%%time
# Your code/ answer goes here.
shape=(6, 7)
winCondition = 3 #how many pieces connected in a row to win


board = empty_board()
move = minimax(board, 'x', 8)
#output results:
visualizeBoard(board)
print(move)

KeyboardInterrupt: 

The above results show that as the size of the board increases (or size of search space increases) the execution time of minimax search increases exponentially. As the size of the board increases the number of children of each space/node increases by a large amount. Meaning, for each extra child alot more function calls/recursion will be needed. The searches are also somewhat depth limited. It would take along time to find a terminal state/loop through all terminal states for a larger board.

### Move ordering

Describe and implement a simple move ordering strategy. How does this strategy influence the time it takes to 
make a move?

In [14]:
# Your code/ answer goes here.


### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

In [15]:
# Your code/ answer goes here.

## Task 4: Heuristic Alpha-Beta Tree Search [3 points] 

### Heuristic evaluation function

Define and implement a heuristic evaluation function.

In [16]:
# Your code/ answer goes here.

### Cutting off search 

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

In [17]:
# Your code/ answer goes here.

Experiment with the same manually created boards as above to check if the agent spots wining opportunities.

In [18]:
# Your code/ answer goes here.

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [19]:
# Your code/ answer goes here.

### Forward Pruning

Add forward pruning to the cutoff search where you do not consider moves that have a low evaluation value after a shallow search 
(way smaller than the cuttoff value).

In [20]:
# Your code/ answer goes here.

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [21]:
# Your code/ answer goes here.

### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function or different forward pruning) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [22]:
# Your code/ answer goes here.

## Challenge task [+ 1 bonus point]

Find another student and let your best agent play against the other student's best player. We will set up a class tournament on Canvas. This tournament will continue after the submission deadline.

## Graduate student advanced task: Pure Monte Carlo Search and Best First Move [1 point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

### Pure Monte Carlos Search

Implement Pure Monte Carlo Search and investigate how this search performs on the test boards that you have used above. 

In [23]:
# Your code/ answer goes here.

### Best First Move

How would you determine what the best first move is? You can use Pure Monte Carlo Search or any algorithms 
that you have implemented above.

In [24]:
# Your code/ answer goes here.